In [45]:
import pydicom as pm
import pandas as pd
import os

In [139]:
def rename_duplicate_columns(df):
    """Rename duplicate columns in the DataFrame."""
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_DUP' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols



In [14]:
Hospital_name= "Guilan"
directory_shortlist=f"D:\Data\Big Pancreas (CT, EUS)\Raw Data Hospital\{Hospital_name}_data_short_just_dcm.xlsx"
directory_longlist=f"D:\Data\Big Pancreas (CT, EUS)\Raw Data Hospital\{Hospital_name}_data.csv"

In [ ]:
directory_longlist=pd.read_csv(directory_longlist)
directory_longlist_dcm=directory_longlist[directory_longlist['If_dicom']==True]
directory_longlist=pd.read_csv(directory_longlist)
directory_longlist_dcm=directory_longlist[directory_longlist['If_dicom']==True]
directory_longlist_dcm=directory_longlist_dcm.reset_index()


In [151]:
import os
import pandas as pd
import pydicom as pm

dcminfo_list = []  # List to store the individual DataFrame pieces

for i in range(1, len(directory_longlist_dcm)):
    dir_path = os.path.join(directory_longlist_dcm.iloc[i][4], directory_longlist_dcm.iloc[i][3])
    ds = pm.dcmread(dir_path)
    ds = pd.DataFrame(ds.values())
    ds[0] = ds[0].apply(lambda x: pm.dataelem.DataElement_from_raw(x) if isinstance(x, pm.dataelem.RawDataElement) else x)
    ds['name'] = ds[0].apply(lambda x: x.name)
    ds['value'] = ds[0].apply(lambda x: x.value)
    ds = ds[['name', 'value']]
    ds = ds.T
    new_header = ds.iloc[0]  # First row as header
    ds = ds[1:]  # Taking the rest of the data
    ds.columns = new_header  # Setting the new header
    ds['to_directory'] = dir_path
    ds['key2csv']=directory_longlist_dcm['Unnamed: 0'][i]
    

    dcminfo_list.append(ds)

for df in dcminfo_list:
    rename_duplicate_columns(df)


dcminfo_all=pd.concat(dcminfo_list, ignore_index=True, sort=False)
dcminfo_all


KeyError: 0